# Data Preprocessing

## Library

In [345]:
import pandas as pd
import re

## Load data

In [346]:
train = pd.read_csv('../data/train.csv')
dev = pd.read_csv('../data/dev.csv')
test = pd.read_csv('../data/test.csv')

## Preprocessing

### 좌우 공백 제거
### 자음 or 모음만으로 구성된 경우 적절한 값으로 대체

In [347]:
train2 = train.copy()
dev2 = dev.copy()
test2 = test.copy()

In [348]:
print(train2.dialogue.apply(lambda x: len(x)).mean())
print(train2.summary.apply(lambda x: len(x)).mean())
print(dev2.dialogue.apply(lambda x: len(x)).mean())
print(dev2.summary.apply(lambda x: len(x)).mean())
print(test2.dialogue.apply(lambda x: len(x)).mean())

438.73083406919807
87.39070402183512
432.5070140280561
81.70941883767534
449.27254509018036


In [349]:
## train ##

# 좌우 공백 제거
train['dialogue'] = train['dialogue'].apply(lambda x: x.strip())
train['summary'] = train['summary'].apply(lambda x: x.strip())
train['topic'] = train['topic'].apply(lambda x: x.strip())

# 자음, 모음만으로 구성된 경우 적절한 값으로 대체
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('제ㅏ', '제가'))
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('척했ㄷ거든', '척했거든'))
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('배경ㅇ로', '배경으로'))
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('ㅋㅋ', '웃기다'))
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('아직ㅍ알맞는', '아직 알맞는'))
train['summary'] = train['summary'].apply(lambda x: x.replace('머라이어 ㅐ리', '머라이어 캐리'))


## dev ##

# 좌우 공백 제거
dev['dialogue'] = dev['dialogue'].apply(lambda x: x.strip())
dev['summary'] = dev['summary'].apply(lambda x: x.strip())
dev['topic'] = dev['topic'].apply(lambda x: x.strip())

## test ##

# 좌우 공백 제거
test['dialogue'] = test['dialogue'].apply(lambda x: x.strip())

In [350]:
print(train2.dialogue.apply(lambda x: len(x)).mean() - train.dialogue.apply(lambda x: len(x)).mean())
print(train2.summary.apply(lambda x: len(x)).mean() - train.summary.apply(lambda x: len(x)).mean())
print(dev2.dialogue.apply(lambda x: len(x)).mean() - dev.dialogue.apply(lambda x: len(x)).mean())
print(dev2.summary.apply(lambda x: len(x)).mean() - dev.summary.apply(lambda x: len(x)).mean())
print(test2.dialogue.apply(lambda x: len(x)).mean() - test.dialogue.apply(lambda x: len(x)).mean())

0.0
0.0
0.0
0.0
0.0


공백을 제거하자 length가 감소하는 것을 알 수 있다.

#### train special tokens 처리

In [351]:
hash_list_1 = []
hash_list_2 = []

for i in range(len(train)):
    hash_list_1.extend(re.findall("#(.*?)#", train.iloc[i, 1]))
    hash_list_2.extend(re.findall("#(.*?)#", train.iloc[i, 2]))

In [352]:
set(hash_list_1)

{'Address',
 'CarNumber',
 'CardNumber',
 'DateOfBirth',
 'Email',
 'PassportNumber',
 'Person1',
 'Person2',
 'Person3',
 'Person4',
 'Person5',
 'Person6',
 'Person7',
 'PhoneNumber',
 'SSN'}

In [353]:
set(hash_list_2)

{'Person1', 'Person2', 'Person3', 'Person4', 'Person6', 'Person7'}

In [354]:
# 찾고자 하는 해시태그
search_tags = ['', 'Person', 'Person 2',
               'PhoneNumber이고, 휴대폰 번호는 ']

# 발견된 행 번호를 저장할 리스트
found_rows = []

for i in range(len(train)):
    # 1번째 열에서 해시태그 찾기
    hashtags_1 = re.findall("#(.*?)#", train.iloc[i, 1])
    # 2번째 열에서 해시태그 찾기
    hashtags_2 = re.findall("#(.*?)#", train.iloc[i, 2])
    
    # 빈 문자열이나 'Person'이 포함되어 있는지 확인
    if any(tag in search_tags for tag in hashtags_1) or any(tag in search_tags for tag in hashtags_2):
        # 조건을 만족하는 경우 행 번호 저장
        found_rows.append(i)

# 결과 출력
print("Found rows:", found_rows)

Found rows: []


In [355]:
train.iloc[420, 1] = re.sub(r'(?<!\#)#PhoneNumber(?!\#)', '#PhoneNumber#', train.iloc[420, 1])
train.iloc[1899, 1] = re.sub(r'#Person#', '#Person2#', train.iloc[1899, 1])
train.iloc[4537, 1] = re.sub(r'#Person 2#', '#Person2#', train.iloc[4537, 1])
train.iloc[11713, 1] = re.sub(r'##Person1#', '#Person1#', train.iloc[11713, 1])

In [356]:
# 찾고자 하는 해시태그
search_tags = ['에게 내일 아침 7시와 7시 30분에 ',
               '은 그 기간 동안 ',
               '은 시험에 대한 준비가 된 것 같다며 ',
               '이 ']

# 발견된 행 번호를 저장할 리스트
found_rows = []

for i in range(len(train)):
    # 1번째 열에서 해시태그 찾기
    hashtags_1 = re.findall("#(.*?)#", train.iloc[i, 1])
    # 2번째 열에서 해시태그 찾기
    hashtags_2 = re.findall("#(.*?)#", train.iloc[i, 2])
    
    # 빈 문자열이나 'Person'이 포함되어 있는지 확인
    if any(tag in search_tags for tag in hashtags_1) or any(tag in search_tags for tag in hashtags_2):
        # 조건을 만족하는 경우 행 번호 저장
        found_rows.append(i)

# 결과 출력
print("Found rows:", found_rows)

Found rows: []


In [357]:
train.iloc[1030, 2] = re.sub(r'사람2#', '#Person2#', train.iloc[1030, 2])
train.iloc[1133, 2] = re.sub(r'사람2#', '#Person2#', train.iloc[1133, 2])
train.iloc[1199, 2] = re.sub(r'사람1#', '#Person1#', train.iloc[1199, 2])

#### dev special tokens 처리

In [358]:
hash_list_3 = []
hash_list_4 = []

for i in range(len(dev)):
    hash_list_3.extend(re.findall("#(.*?)#", dev.iloc[i, 1]))
    hash_list_4.extend(re.findall("#(.*?)#", dev.iloc[i, 2]))

In [359]:
set(hash_list_3) # dialogue는 괜찮음

{'Address',
 'DateOfBirth',
 'Person1',
 'Person2',
 'Person3',
 'Person4',
 'PhoneNumber'}

In [360]:
set(hash_list_4) # summary는 수정 필요함

{'Person1', 'Person2', 'Person3'}

In [361]:
# 찾고자 하는 해시태그
search_tags = ['사람1']

# 발견된 행 번호를 저장할 리스트
found_rows = []

for i in range(len(dev)):
    # 1번째 열에서 해시태그 찾기
    hashtags_3 = re.findall("#(.*?)#", dev.iloc[i, 1])
    # 2번째 열에서 해시태그 찾기
    hashtags_4 = re.findall("#(.*?)#", dev.iloc[i, 2])
    
    # 빈 문자열이나 'Person'이 포함되어 있는지 확인
    if any(tag in search_tags for tag in hashtags_3) or any(tag in search_tags for tag in hashtags_4):
        # 조건을 만족하는 경우 행 번호 저장
        found_rows.append(i)

# 결과 출력
print("Found rows:", found_rows)

Found rows: []


In [362]:
dev.iloc[43, 2] = re.sub(r'#사람1#', '#Person1#', dev.iloc[43, 2])

#### test는 special tokens가 잘 처리 되어 있었음

In [363]:
test

,fname,dialogue
0,test_0,"#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, ..."
1,test_1,#Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?\n#Person2#: 또...
2,test_2,"#Person1#: 케이트, 무슨 일이 일어났는지 너는 믿지 못할거야. \n#Per..."
3,test_3,"#Person1#: 생일 축하해, 이건 너를 위한 거야, 브라이언.\n#Person..."
4,test_4,#Person1#: 이 올림픽 공원이 정말 크네요!\n#Person2#: 네. 지금...
...,...,...
494,test_495,"#Person1#: 헤이, 찰리, 학교 끝나고 우리 집에 와서 나랑 비디오 게임 할..."
495,test_496,#Person1#: 어떻게 컨트리 음악에 관심을 가지게 되었나요?\r\n#Perso...
496,test_497,"#Person1#: 실례합니다, 앨리스. 이곳을 사용해본 적이 없는데, 기계를 어떻..."
497,test_498,#Person1#: 매튜? 안녕!\r\n#Person2#: 스티브! 오랜만이네! 얼...


In [364]:
hash_list_5 = []

for i in range(len(dev)):
    hash_list_5.extend(re.findall("#(.*?)#", test.iloc[i, 1]))

In [365]:
set(hash_list_5)

{'Address', 'PassportNumber', 'Person1', 'Person2', 'Person3', 'PhoneNumber'}

In [366]:
# 동일한 화자가 연속으로 말하는 경우

def find_consecutive_speakers(dialogue):
    speaker_pattern = re.compile(r'#Person\d+#')
    speakers = speaker_pattern.findall(dialogue)
    consecutive_speakers = []
    for i in range(1, len(speakers)):
        if speakers[i] == speakers[i - 1]:
            consecutive_speakers.append((i - 1, i, speakers[i]))
    return consecutive_speakers

# 잘못된 화자 이름 예) 사람1
def find_misspelled_speakers(dialogue):
    incorrect_speaker_pattern = re.compile(r'(사람\d+|PERSON\d+)')
    misspelled_speakers = incorrect_speaker_pattern.findall(dialogue)
    return misspelled_speakers

# 모든 오류 찾기
for index, row in train.iterrows():
    consecutive_speakers = find_consecutive_speakers(row['dialogue'])
    misspelled_speakers = find_misspelled_speakers(row['dialogue'])
    
    if consecutive_speakers or misspelled_speakers:
        print(f"Index {index}:")
        if consecutive_speakers:
            print(f"  Consecutive speakers: {consecutive_speakers}")
        if misspelled_speakers:
            print(f"  Misspelled speakers: {misspelled_speakers}")


Index 345:
  Consecutive speakers: [(12, 13, '#Person1#'), (13, 14, '#Person1#')]
Index 484:
  Consecutive speakers: [(4, 5, '#Person1#')]
Index 756:
  Consecutive speakers: [(0, 1, '#Person1#')]
Index 839:
  Consecutive speakers: [(7, 8, '#Person2#')]
  Misspelled speakers: ['사람1']
Index 872:
  Consecutive speakers: [(2, 3, '#Person1#'), (3, 4, '#Person1#')]
Index 925:
  Consecutive speakers: [(2, 3, '#Person1#')]
Index 982:
  Consecutive speakers: [(1, 2, '#Person2#')]
Index 1033:
  Consecutive speakers: [(3, 4, '#Person2#')]
Index 1125:
  Misspelled speakers: ['사람1']
Index 1142:
  Misspelled speakers: ['사람1']
Index 1220:
  Consecutive speakers: [(3, 4, '#Person2#'), (4, 5, '#Person2#')]
Index 1236:
  Consecutive speakers: [(6, 7, '#Person1#')]
Index 1278:
  Consecutive speakers: [(2, 3, '#Person1#')]
Index 1283:
  Consecutive speakers: [(1, 2, '#Person2#')]
Index 1294:
  Consecutive speakers: [(4, 5, '#Person1#')]
Index 1301:
  Consecutive speakers: [(5, 6, '#Person2#')]
Index 1302:

In [367]:
num = 11578
print(train.iloc[num, 1])
print('-'*30)
print(train.iloc[num, 2])

#Person1#: 쿠퍼씨! 어제 수업에서 가르쳐주신 것을 바탕으로 제 스스로 색을 만들어 보려고 했습니다. 그런데 쉽지 않았어요.
#Person1#: 여기 보시겠어요? 보라색을 만들고 싶었는데, 파란색을 충분히 사용하지 않아서 약간 회색이고 지루하게 나왔어요.
#Person2#: 그래도 잘 만들어진 것 같아요! 이게 바로 그림 그리는 재미 중 하나죠 - 조금의 상상력으로 할 수 있는 것을 발견하고, 위험을 감수하고, 새로운 것을 시도하는 것.
#Person1#: 격려해주셔서 감사합니다.
#Person2#: 때때로 가장 아름다운 것들은 예술적인 우연에서 나옵니다. 그리고 저는 거의 모든 흥미로운 것들도 마찬가지라고 생각해요.
------------------------------
쿠퍼 선생님은 #Person1#의 그림을 칭찬하며 격려하고, 가장 아름다운 것들은 예술적인 우연에서 나온다고 #Person1#에게 말한다


In [368]:
train.iloc[345, 1] = re.sub(r'#Person1#: 아니요. 입원할 필요는 없습니다.', '#Person2#: 아니요. 입원할 필요는 없습니다.', train.iloc[345, 1])
train.iloc[484, 1] = re.sub(r'#Person1#: 인상적이네. 우리는 좋은 관계야.', '#Person1#: 인상적이네. \n#Person2#: 우리는 좋은 관계야.', train.iloc[484, 1])
train.iloc[756, 1] = re.sub(r'#Person1#: 안녕, 메이슨!', '#Person2#: 안녕, 메이슨!', train.iloc[756, 1])
train.iloc[839, 1] = re.sub(r'#사람1', '#Person1#: ', train.iloc[839, 1])
train.iloc[872, 1] = re.sub(r'#Person1#: 중국은행에서', '#Person2#: 중국은행에서', train.iloc[872, 1])
train.iloc[925, 1] = re.sub(r'\n#Person1#: 적자생존이라는', ' 적자생존이라는', train.iloc[925, 1])
train.iloc[982, 1] = re.sub(r'\n#Person2#: 왜 안 가세요', ' 왜 안 가세요', train.iloc[982, 1])
train.iloc[1033, 1] = re.sub(r' 메리의 취미가 뭐야?', '\n#Person1#: 메리의 취미가 뭐야?', train.iloc[1033, 1])
train.iloc[1125, 1] = re.sub(r'사람1#', '#Person1#', train.iloc[1125, 1])
train.iloc[1142, 1] = re.sub(r'사람1#', '#Person1#', train.iloc[1142, 1])
train.iloc[1220, 1] = re.sub(r'#Person2#: 음', '#Person1#: 음', train.iloc[1220, 1])
train.iloc[1236, 1] = re.sub(r'#고객님', '#Person2#: 고객님', train.iloc[1236, 1])
train.iloc[1278, 1] = re.sub(r'#고객님', '#Person2#: 고객님', train.iloc[1278, 1])
train.iloc[1283, 1] = re.sub(r'#어디', '#Person1#: 어디', train.iloc[1283, 1])

train.iloc[1301, 1] = re.sub(r'#샐러드용', '#Person1#: 샐러드용', train.iloc[1301, 1])
train.iloc[1302, 1] = re.sub(r'#페리에와', '#Person1#: 페리에와', train.iloc[1302, 1])
train.iloc[1306, 1] = re.sub(r'#나', '#Person2#: 나', train.iloc[1306, 1])
train.iloc[1322, 1] = re.sub(r'#여기서', '#Person1#: 여기서', train.iloc[1322, 1])
train.iloc[1419, 1] = re.sub(r'\n#Person1#: 어떠세요?', ' 어떠세요?', train.iloc[1419, 1])
train.iloc[1424, 1] = re.sub(r' 방으로 가는 길은 어느 쪽인가요?', '#Person1#: 방으로 가는 길은 어느 쪽인가요?', train.iloc[1424, 1])
train.iloc[1440, 1] = re.sub(r' 언제 예약하시겠습니까', '\n#Person1#: 언제 예약하시겠습니까', train.iloc[1440, 1])
train.iloc[1475, 1] = re.sub(r'\n#Person2#: 그거 좋겠네요.', '', train.iloc[1475, 1])
train.iloc[1497, 1] = re.sub(r' 복사 한', '#Person1#: 복사 한', train.iloc[1497, 1])
train.iloc[1547, 1] = re.sub(r'#작은 걸로 주세요.', '#Person2#: 작은 걸로 주세요.', train.iloc[1547, 1])
train.iloc[1791, 1] = re.sub(r'#Person1#: 봐', '#Person2#: 봐', train.iloc[1791, 1])
train.iloc[3628, 1] = re.sub(r' 당신의 전면 범퍼도', '\n#Person1#: 당신의 전면 범퍼도', train.iloc[3628, 1])
## 5441은 두 번 반복됨 ##
train.iloc[5441, 1] = '#Person1#: 안녕, 내가 왔어! 음. . . 너의 머리가 꽤 손상되었고, 끝이 갈라져있어.\n#Person2#: 정말? 어떻게 해야 할까?\n#Person1#: 잘라내면 갈라진 끝은 해결될 거야, 하지만 딥 컨디셔닝 트리트먼트가 필요할 수도 있어.\n#Person2#: 음, 알겠어. 네가 도움이 될 것 같다고 생각하는 대로 해줘.\n#Person1#: 얼굴을 감싸는 레이어도 추가할 거야.\n#Person2#: 위쪽으로 가위로 손을 좀 봐줄 수 있을까? 내 머리 숫이 정말 많아.\n#Person1#: 문제 없어!'

train.iloc[6759, 1] = re.sub(r'비위생적이져죠', '비위생적이죠', train.iloc[6759, 1])
train.iloc[6759, 1] = re.sub(r'\n#Person2#: 아,', '아,', train.iloc[6759, 1])
train.iloc[6799, 1] = re.sub(r'\n#Person1#: 뭐, 음.', ' 뭐, 음.', train.iloc[6799, 1])
train.iloc[8645, 1] = re.sub(r'#Person2#: 저는 서프라이즈', '#Person1#: 저는 서프라이즈', train.iloc[8645, 1])
train.iloc[9898, 1] = re.sub(r'#Person1#: 오, 안녕', '#Person2#: 오, 안녕', train.iloc[9898, 1])
train.iloc[11578, 1] = re.sub(r'그런데 쉽지 않았어요.\r\n#Person1#: 여기', '그런데 쉽지 않았어요. 여기', train.iloc[11578, 1])
train.drop(1294, inplace=True)
train.reset_index(drop=True, inplace=True)

In [369]:
train.to_csv('../data/train.csv', index=False)
dev.to_csv('../data/dev.csv', index=False)
test.to_csv('../data/test.csv', index=False)